In [1]:
import torch as pt
import torch.nn as nn
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

from torch.distributions.multivariate_normal import MultivariateNormal
from torch.distributions.uniform import Uniform

from networks import PiNetTV, QNetTV

In [2]:
# Parameters

horizon = 5

## LQR Optimality Benchmark

Below we solve the LQR control problem exactly using CVXPY. First, set up the dynamics and costs. The goal is to navigate from $(2, 3)$ to origin.

In [3]:
A = np.array([[1, 1], [0, 1]])
B = np.array([[0], [1]])

def dynamics(x, u):
    return A @ x + B @ u

def cost(x, u):
    return cp.sum_squares(x)

def terminal_cost(x):
    return 100 * cost(x, np.array([[0]]))

Now actually perform the optimization:

In [4]:
x0 = np.array([2.0, 3.0])

x = cp.Variable((2, horizon + 1))
u = cp.Variable((1, horizon))

cstr = [x[:, 0] == x0]
cost_func = 0

for t in range(horizon):
    cost_func += cost(x[:, t], u[:, t])
    cstr += [x[:, t + 1] == dynamics(x[:, t], u[:, t])]
    
cost_func += terminal_cost(x[:, -1])

problem = cp.Problem(cp.Minimize(cost_func), cstr)

print(f'Optimal value: {problem.solve()}')
print('Optimal trajectory:')
print(x.value.round(decimals=4))

Optimal value: 53.475627070515856
Optimal trajectory:
[[ 2.0000e+00  5.0000e+00  1.9049e+00  7.1460e-01  2.3900e-01  2.4000e-03]
 [ 3.0000e+00 -3.0951e+00 -1.1903e+00 -4.7560e-01 -2.3660e-01  0.0000e+00]]


## Policy Gradient

Now we will learn a solution to the LQR problem via policy gradient. Below outlines two approaches. Both output Gaussian samples from distributions depending on the input state, but one outputs they differ in how they specify the distribution. The first model, `Controller`, outputs the diagonal of a matrix $A$ that specifies the covariance of the output as $AA^{\mathrm{T}}$. The second outputs the log of the diagonal of the covariance matrix directly. The performance of these two parameterizations will be compared below.

In [5]:
ntrvs = 2

def make_q_net(t):
    return nn.Sequential(nn.Linear(4, 64), 
                         nn.ELU(), 
                         nn.Linear(64, 64), 
                         nn.ELU(), 
                         nn.Linear(64, ntrvs * 2))

def make_pi_net(t):
    return nn.Sequential(nn.Linear(ntrvs, 64), 
                         nn.ELU(), 
                         nn.Linear(64, 64), 
                         nn.ELU(), 
                         nn.Linear(64, 2))

The following cell defines the function that will actually train our models.

In [8]:
def rollout(pi_net, q_net, scenario, batch_size):
    states = []
    outputs = []
    trvs = []
    inputs = []
    costs = []

    log_probs = pt.zeros((horizon, batch_size))

    for s in range(batch_size):
        traj_states = [scenario.sample_initial_dist().reshape((-1, 1))]
        traj_outputs = []
        traj_trvs = []
        traj_inputs = []
        traj_costs = []
        
        trv = pt.zeros(ntrvs).reshape((-1, 1))

        for t in range(horizon):
            traj_outputs.append(scenario.sensor(traj_states[-1].flatten(), t).reshape((-1, 1)))            
            
            trv = q_net(traj_outputs[t].flatten(), trv.flatten(), t).reshape((-1, 1))
            traj_trvs.append(trv)
            
            traj_inputs.append(pi_net(traj_trvs[t].flatten(), t).reshape((-1, 1)))
            traj_costs.append(scenario.cost(traj_states[t].flatten(), traj_inputs[t].flatten(), t).reshape((-1, 1)))
            traj_states.append(scenario.dynamics(traj_states[t].flatten(), traj_inputs[t], t).reshape((-1, 1)).detach())

        traj_costs.append(scenario.terminal_cost(traj_states[-1].flatten()).reshape((-1, 1)))

        states.append(pt.cat(traj_states, axis=1))
        outputs.append(pt.cat(traj_outputs, axis=1))        
        trvs.append(pt.cat(traj_trvs, axis=1))
        inputs.append(pt.cat(traj_inputs, axis=1))
        costs.append(pt.cat(traj_costs, axis=1))
    
    states = pt.stack(states, axis=2).detach()
    outputs = pt.stack(outputs, axis=2).detach()
    trvs = pt.stack(trvs, axis=2)
    inputs = pt.stack(inputs, axis=2).detach()
    costs = pt.stack(costs, axis=2)[0, :, :].detach()
        
    return states, outputs, trvs, inputs, costs

def train(pi_net, q_net, scenario, epochs, batch_size, lr):
    opt = pt.optim.Adam(list(pi_net.parameters()) + list(q_net.parameters()), lr=lr)

    values = np.zeros(epochs)
    
    for epoch in range(epochs):                
        pi_log_probs = pt.zeros((horizon, batch_size))
        q_log_probs = pt.zeros((horizon, batch_size))
        
        states, outputs, trvs, inputs, costs = rollout(pi_net, q_net, scenario, batch_size)        
        values[epoch] = costs.sum(axis=0).mean()
        
        print(f'[{epoch}]\t\t{values[epoch]}') 
        
        for s in range(batch_size):
            trv = pt.zeros(ntrvs)
            
            for t in range(horizon):         
                q_log_probs[t, s] = q_net.log_prob(trvs[:, t, s].detach(), outputs[:, t, s], trv.detach(), t)
                pi_log_probs[t, s] = pi_net.log_prob(inputs[:, t, s], trvs[:, t, s], t)
                trv = trvs[:, t, s]
                
        baseline = costs.sum(axis=0).mean()
                
        opt.zero_grad()        
        loss = pt.mul(pi_log_probs.sum(axis=0), costs.sum(axis=0) - baseline).mean() + \
               pt.mul(q_log_probs.sum(axis=0), costs.sum(axis=0) - baseline).mean()
        loss.backward()
        
        opt.step()
        
        if epoch == epochs - 1:
            return states, outputs, trvs, inputs, costs, values
        

The following cells will train the two models with the same hyperparameters and plot a smoothed version of their loss function over the training process. Note that other than the randomness of the control policy, the dyanmics of the system are deterministic here.

In [9]:
from scenarios import LQRScenario, LavaScenario

init_dist = Uniform(0, 5)
sensor_dist = MultivariateNormal(pt.tensor([0.0, 0.0]), 0.01 * pt.eye(2))

scenario = LavaScenario(lambda: init_dist.sample().item(), lambda: sensor_dist.sample())

pt.manual_seed(0)

pi_net = PiNetTV(make_pi_net, horizon)
q_net = QNetTV(make_q_net, horizon)
states, outputs, trvs, inputs, costs, values = train(pi_net, q_net, scenario, epochs=200, batch_size=100, lr=0.0005)

[0]		863.9296264648438
[1]		844.318603515625
[2]		756.8457641601562
[3]		687.4397583007812
[4]		707.6642456054688
[5]		644.6332397460938
[6]		757.3072509765625
[7]		791.5577392578125
[8]		672.6542358398438
[9]		747.8230590820312
[10]		648.2454833984375
[11]		752.33056640625
[12]		674.8875732421875
[13]		736.9977416992188
[14]		661.3593139648438
[15]		731.8026733398438
[16]		629.6526489257812
[17]		650.447021484375
[18]		636.896240234375
[19]		690.139404296875
[20]		727.3494262695312
[21]		670.5134887695312
[22]		639.4796752929688
[23]		603.8712768554688
[24]		693.7208862304688
[25]		657.169921875
[26]		637.8692016601562
[27]		623.0638427734375
[28]		665.5797119140625
[29]		629.7118530273438
[30]		606.3330688476562
[31]		576.2198486328125
[32]		661.5497436523438
[33]		536.869873046875
[34]		538.1593627929688
[35]		570.0509643554688
[36]		687.7838134765625
[37]		667.2703857421875
[38]		601.0037231445312
[39]		573.6373901367188
[40]		542.6990356445312
[41]		598.563232421875
[42]		586.6234

In [ ]:
# Taken from Tensorboard
def smooth(scalars, weight):
    last = scalars[0]
    smoothed = list()
    for point in scalars:
        smoothed_val = last * weight + (1 - weight) * point
        smoothed.append(smoothed_val)
        last = smoothed_val

    return np.array(smoothed)

smoothed_values = smooth(lvalues, 0.8)

plt.plot(smoothed_values, label='Log Stds')
plt.legend()

As you can see, the log model's loss function converges more quickly than the other model. We now repeat this experiment, but with the initial condition of the system perturbed by noise drawn from $\mathcal{N}(0, I)$ at the start of each rollout.

In [ ]:
states.mean(axis=2)